In [1]:
from kafka import KafkaProducer, KafkaConsumer
import json
import os

In [2]:
bootstraps = ['ip-20-0-31-210.ec2.internal:9092', 
              'ip-20-0-31-221.ec2.internal:9092', 
              'ip-20-0-31-221.ec2.internal:9092']

In [3]:
cwd = os.getcwd()
for part in cwd.split('/'):
    if part.lower().startswith('edureka'):
        topic_name = part.lower()
topic_name
topics = [topic_name]

In [4]:
consumer = KafkaConsumer(bootstrap_servers=bootstraps,
                         auto_offset_reset='earliest',
                         api_version=(0, 10),
                         consumer_timeout_ms=1000)

json_consumer = KafkaConsumer(bootstrap_servers=bootstraps,
                              auto_offset_reset='earliest',
                              api_version=(0, 10),
                              consumer_timeout_ms=1000,
                              value_deserializer=lambda m: json.dumps(m.decode('ascii'))
                             )

In [5]:
json_consumer.subscribe(topics)

In [6]:
import pyspark
spark = SparkSession.builder.appName("SpamDetection Notebook").getOrCreate()
from pyspark.ml import PipelineModel
pipeline = PipelineModel.load("use_cases/spam_model4.4")


In [7]:
d={}
for message in json_consumer:
    # message value and key are raw bytes -- decode if necessary!
    # e.g., for unicode: message.value.decode('utf-8')
    print ("%s:%d:%d: key=%s value=%s" % (message.topic, 
                                          message.partition,
                                          message.offset, 
                                          message.key,
                                          message.value))
    d['messgae']=message.value
    df=spark.createDataFrame(d)
    prediction=pipeline.transform(df)
    print(prediction)
    

edureka_396003:0:1071: key=None value="{\"foo\": \"bar\"}"


TypeError: Can not infer schema for type: <type 'str'>

In [7]:
consumer.close()
json_consumer.close()

<font color='gray'>References: </font> [Kafka-Python Documentation](https://media.readthedocs.org/pdf/kafka-python/master/kafka-python.pdf "Kafka-Python Documentation")